In [ ]:
import os
from collections import namedtuple

import openpyxl

In [ ]:
output_folders = [os.path.abspath(i)
                  for i in os.listdir('.')
                  if os.path.isdir(os.path.abspath(i)) and '_output' in i]

xlsx_files = [os.path.join(folder, file)
             for folder in output_folders
             for file in os.listdir(folder)
             if os.path.splitext(file)[1] in ('.xlsx', )]
xlsx_files.extend([os.path.join(os.path.abspath('cambridge_output'), file)
                   for file in os.listdir('cambridge_output')
                   if os.path.splitext(file)[1] == ''])

filename_parsedobject = dict()

openpyxl.war
for file in xlsx_files:
    print(file)
    read_file = openpyxl.load_workbook(file)
    print(read_file.sheetnames)
    for sheet in read_file.sheetnames:
        read_sheet = read_file.get_sheet_by_name(sheet)
        for num, row in enumerate(read_sheet.iter_rows()):
            if num == 0:
                headers = (i.value.replace(' ', '_').replace('(', '').replace(')', '').replace('/', '') for i in row)
                WorkSheet = namedtuple('WorkSheet', headers)
                continue
            values = (i.value for i in row)
            item = WorkSheet(*values)
            filename_parsedobject[sheet] = item
  